In [23]:
from osgeo import gdal, ogr, osr
import glob, os, sys, time
import subprocess as sp
# xml realted libraries
from xml.etree import ElementTree
from xml.dom import minidom
from xml.etree.ElementTree import Element, SubElement, Comment
import re

In [2]:
# cmd function, input is a comma separated list of items
def cmd(command):
    print (sp.list2cmdline(command))
    norm = sp.Popen(sp.list2cmdline(command),stdout=sp.PIPE, shell=True).communicate()    
    return norm

In [3]:
def prettify(elem):
    """Return a pretty-printed XML string for the Element.
    """
    rough_string = ElementTree.tostring(elem, 'utf-8')
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent="  ")

def writeOGRVRT(basename, fullpath, x='field_1', y='field_2', z='field_3'):
    top = Element('OGRVRTDataSource')
    child = SubElement(top, 'OGRVRTLayer')
    child.set('name', basename)
    sub_child_1 = SubElement(child, 'SrcDataSource')
    sub_child_1.text = fullpath
    sub_child_2 = SubElement(child, 'GeometryType')
    sub_child_2.text = 'wkbPoint'
    sub_child_3 = SubElement(child, 'LayerSRS')
    sub_child_3.text = 'EPSG:4326'
    sub_child_4 = SubElement(child, 'GeometryField')
    sub_child_4.set('encoding','PointFromColumns')
    sub_child_4.set('x',x)
    sub_child_4.set('y',y)
    sub_child_4.set('z',z)

    return prettify(top)

In [4]:
ogr2ogr = r'C:\Python35\Lib\site-packages\osgeo//ogr2ogr.exe'
gdalwarp = r'C:\Python35\Lib\site-packages\osgeo//gdalwarp.exe'
gdal_rasterize = r'C:\Python35\Lib\site-packages\osgeo//gdal_rasterize.exe'
gdal_translate = r'C:\Python35\Lib\site-packages\osgeo//gdal_translate.exe'
gdalbuildvrt = r'C:\Python35\Lib\site-packages\osgeo//gdalbuildvrt.exe'
gdalinfo = r'C:\Python35\Lib\site-packages\osgeo\gdalinfo.exe'
gdal_grid = r'C:\Python35\Lib\site-packages\osgeo\gdal_grid.exe'
gdal_contour = r'C:\Python35\Lib\site-packages\osgeo\gdal_contour.exe'
ogrinfo = r'C:\Python35\Lib\site-packages\osgeo\ogrinfo.exe'

In [5]:
baseDir = r'D:\Projects\Pr\3412.10'
workDir = os.path.join(baseDir,r'tmp_workdir')
db3Path = os.path.join(baseDir, r'1491383868603_BOAT.db3')

In [6]:
# request info ogr table
cmd_out = cmd([ogrinfo, db3Path])
print (cmd_out[0].decode("utf-8") )

C:\Python35\Lib\site-packages\osgeo\ogrinfo.exe D:\Projects\Pr\3412.10\1491383868603_BOAT.db3
INFO: Open of `D:\Projects\Pr\3412.10\1491383868603_BOAT.db3'
      using driver `SQLite' successful.
1: IceHoles (None)
2: Location (None)
3: Measurement (None)
4: Sonar (None)
5: android_metadata (None)
6: sqlite_sequence (None)



In [8]:
# get info Measurement table
# request info ogr table
cmd_out = cmd([ogrinfo, db3Path,"Measurement"])
print (cmd_out[0].decode("utf-8") )

C:\Python35\Lib\site-packages\osgeo\ogrinfo.exe D:\Projects\Pr\3412.10\1491383868603_BOAT.db3 Measurement
INFO: Open of `D:\Projects\Pr\3412.10\1491383868603_BOAT.db3'
      using driver `SQLite' successful.

Layer name: Measurement
Geometry: None
Feature Count: 965
Layer SRS WKT:
(unknown)
FID Column = _id
latitude: Real (0.0)
longitude: Real (0.0)
depth: Real (0.0)
frequency: Integer (0.0)
weed: Real (0.0)
fish: Integer (0.0)
OGRFeature(Measurement):1
  latitude (Real) = 52.2938220755071
  longitude (Real) = 5.15512679154429
  depth (Real) = 3.64154005050659
  frequency (Integer) = 2
  weed (Real) = 0.109520003199577
  fish (Integer) = 0

OGRFeature(Measurement):2
  latitude (Real) = 52.293841378685
  longitude (Real) = 5.15511862540515
  depth (Real) = 3.97010016441345
  frequency (Integer) = 2
  weed (Real) = 0.410700023174286
  fish (Integer) = 0

OGRFeature(Measurement):3
  latitude (Real) = 52.2938618097673
  longitude (Real) = 5.15511259220717
  depth (Real) = 4.07961988449097


In [9]:
# create unique ID based on modified time when point-db-file is uploaded
tmod = os.path.getmtime(db3Path)            
t = time.strftime('%Y%m%d%H%M%S', time.localtime(int(tmod)))
# db3 file table to look at is 'Measurement'

In [ ]:
# import base64
# import uuid

# # get a UUID - URL safe, Base64
# def get_a_uuid():
#     r_uuid = base64.urlsafe_b64encode(uuid.uuid4().bytes)
#     return r_uuid.decode('utf-8').replace('=','')
# get_a_uuid()

In [10]:
# 2 build OGRVRT from CSV file
b = os.path.join(workDir,t+'.vrt')
with open(b, 'w') as the_file:
    the_file.write(writeOGRVRT(basename='Measurement',fullpath=db3Path,x='longitude',y='latitude',z='depth'))

# what is written?    
print(writeOGRVRT(basename='Measurement',fullpath=db3Path,x='longitude',y='latitude',z='depth'))

<?xml version="1.0" ?>
<OGRVRTDataSource>
  <OGRVRTLayer name="Measurement">
    <SrcDataSource>D:\Projects\Pr\3412.10\1491383868603_BOAT.db3</SrcDataSource>
    <GeometryType>wkbPoint</GeometryType>
    <LayerSRS>EPSG:4326</LayerSRS>
    <GeometryField encoding="PointFromColumns" x="longitude" y="latitude" z="depth"/>
  </OGRVRTLayer>
</OGRVRTDataSource>



In [17]:
extent_byte = cmd([ogrinfo,b,'Measurement','|','grep','Extent'])

C:\Python35\Lib\site-packages\osgeo\ogrinfo.exe D:\Projects\Pr\3412.10\tmp_workdir\20170406145141.vrt Measurement | grep Extent


In [24]:
extent_string = extent_byte[0].decode("utf-8") 
re.findall(r"[-+]?\d*\.\d+|\d+", extent_string)

['5.152849', '52.293583', '5.155472', '52.295716']

In [11]:
# # including clip
# wktString = r"POLYGON((5.598607063293457 52.948172441749286, 5.6000661849975595 52.947888001813055, 5.600194931030274 52.94729325772072, 5.5998945236206055 52.94623304144963, 5.5989933013916025 52.94556069571643, 5.597491264343263 52.94540553752482, 5.595645904541016 52.94574171290331, 5.595088005065919 52.947163964444705, 5.59680461883545 52.94700881200339, 5.597577095031739 52.94662092846549, 5.598478317260742 52.94690537673336, 5.597662925720216 52.947577701567695, 5.597105026245117 52.94850859562616, 5.598607063293457 52.948172441749286)),4326"
# c = os.path.join(workDir,t+'grid_new1.tif')
# cmd([gdal_grid,'-clipsrc',wktString,b,c])

# excluding clip
c = os.path.join(workDir,t+'grid_new1.tif')
cmd([gdal_grid,b,c])

C:\Python35\Lib\site-packages\osgeo\gdal_grid.exe D:\Projects\Pr\3412.10\tmp_workdir\20170406145141.vrt D:\Projects\Pr\3412.10\tmp_workdir\20170406145141grid_new1.tif


(b'Grid data type is "Float64"\r\nGrid size = (256 256).\r\nCorner coordinates = (5.152844 52.295720)-(5.155478 52.293579).\r\nGrid cell size = (0.000010 0.000008).\r\nSource point count = 965.\r\nAlgorithm name: "invdist".\r\nOptions are "power=2.000000:smoothing=0.000000:radius1=0.000000:radius2=0.000000:angle=0.000000:max_points=0:min_points=0:nodata=0.000000"\r\n\r\n0...10...20...30...40...50...60...70...80...90...100 - done.\r\n',
 None)

In [12]:
# db3-points to GeoJSON-points
d = os.path.join(workDir,t+'pts.json')
cmd([ogr2ogr, '-f','GeoJSON',d,b])

C:\Python35\Lib\site-packages\osgeo//ogr2ogr.exe -f GeoJSON D:\Projects\Pr\3412.10\tmp_workdir\20170406145141pts.json D:\Projects\Pr\3412.10\tmp_workdir\20170406145141.vrt


(b'', None)

In [ ]:
polygonPath  = os.path.join(baseDir, r'leaflet_draw','clip_indijk.geojson')
e = os.path.join(workDir,t+'grid_cutline3.tif')
cmd([gdalwarp,'-cutline', polygonPath,'-crop_to_cutline','-dstalpha','-dstnodata','-999',c,e])